# Expectation-Maximization Clustering

**Maximization step**
$$
q_{mk} = \frac{
            \sum_{n=1}^{N} r_{nk}I(t_m \in d_n)
            }{
            \sum_{n=1}^{N} r_{nk}
            }
            \ ;\ \ \alpha_k = \frac{\sum_{n=1}^{N} r_{nk}}{N}
$$
**Expectation step**
$$
r_{nk} = \frac{
            \alpha_k(\prod_{t_m \in d_n} q_{mk})(\prod_{t_m \not\in d_n} (1-q_{mk}))
            }{
            \sum_{k=1}^{K}\alpha_k(\prod_{t_m \in d_n} q_{mk})(\prod_{t_m \not\in d_n} (1-q_{mk}))
            }
$$

where $I(t_m \in d_m) = 1$ if $t_m \in d_m$, $0$ otherwise. $r_{nk}$ is the soft assignment of $d_n$ to $\omega_k$.

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from IPython.display import display, display_html

- `M` one-hot encoding of docs (multivariate Bernoulli distributions for data)
- `N` corpus size
- `K` number of clusters to find
- `m` vocabulary size
- `A` cluster priors
- `Q` tokens assignment to clusters
- `R` documents assignment to clusters

In [2]:
corpus = [
    "sugar milk eggs butter".split(),
    "vanilla eggs apple peach sugar".split(),
    "peach apple sugar milk".split(),
    "meat chicken olive salt butter".split(),
    "meat beef salt pepper".split()
]
vocabulary = list(set([x for y in corpus for x in y]))
M = np.array([[1 if x in y else 0 for x in vocabulary] for y in corpus])
N, K, m = len(corpus), 2, len(vocabulary)
A = np.zeros(K)
Q = np.zeros((m, K))

## Random init $r_{nk}$ (assignment of documents to clusters)

In [3]:
R = np.random.uniform(size=(N, K))
R = R / R.sum(axis=1).reshape(-1, 1)

In [4]:
R

array([[0.33059267, 0.66940733],
       [0.48753123, 0.51246877],
       [0.62628952, 0.37371048],
       [0.32633183, 0.67366817],
       [0.53856354, 0.46143646]])

## Maximization step

$$
q_{mk} = \frac{
            \sum_{n=1}^{N} r_{nk}I(t_m \in d_n)
            }{
            \sum_{n=1}^{N} r_{nk}
            }
            \ ;\ \ \alpha_k = \frac{\sum_{n=1}^{N} r_{nk}}{N}
$$


In [5]:
R[:,0] * M[:,2]

array([0.        , 0.        , 0.        , 0.32633183, 0.        ])

In [6]:
def qmk(token_index, cluster_index):
    num = (R[:,cluster_index] * M[:,token_index]).sum()
    den = R[:,cluster_index].sum()
    return num / den

def ak(cluster_index):
    return R[:,cluster_index].sum() / N

In [7]:
qmk(2, 0)

0.14131147580065537

In [8]:
ak(0)

0.4618617595694018

In [9]:
def maxstep(Q, A):
    for cluster_index in range(K):
        for token_index in range(len(vocabulary)):
            Q[token_index, cluster_index] = qmk(token_index, cluster_index)
        A[cluster_index] = ak(cluster_index)
    return Q / Q.sum(axis=1).reshape(-1, 1), A

In [10]:
Adf = pd.DataFrame(A)
Qdf = pd.DataFrame(Q, index=vocabulary).T
A_styler = Adf.style.set_table_attributes("style='display:inline'").set_caption('A')
Q_styler = Qdf.style.set_table_attributes("style='display:inline'").set_caption('Q')
display_html(A_styler._repr_html_() + Q_styler._repr_html_(), raw=True)

Q, A = maxstep(Q, A)

Adf = pd.DataFrame(A)
Qdf = pd.DataFrame(Q, index=vocabulary).T
A_styler = Adf.style.set_table_attributes("style='display:inline'").set_caption('A')
Q_styler = Qdf.style.set_table_attributes("style='display:inline'").set_caption('Q')
display_html(A_styler._repr_html_() + Q_styler._repr_html_(), raw=True)

A 
 
 
   
 0 
 
 
 
 
 0 
 0.000000 
 
 
 1 
 0.000000 
 
 
 

 
 Q 
 
 
   
 peach 
 chicken 
 olive 
 beef 
 pepper 
 butter 
 sugar 
 eggs 
 salt 
 vanilla 
 milk 
 meat 
 apple 
 
 
 
 
 0 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 
 
 1 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000

A 
 
 
   
 0 
 
 
 
 
 0 
 0.461862 
 
 
 1 
 0.538138 
 
 
 

 
 Q 
 
 
   
 peach 
 chicken 
 olive 
 beef 
 pepper 
 butter 
 sugar 
 eggs 
 salt 
 vanilla 
 milk 
 meat 
 apple 
 
 
 
 
 0 
 0.594231 
 0.360782 
 0.360782 
 0.576253 
 0.576253 
 0.363016 
 0.519665 
 0.446457 
 0.470280 
 0.525718 
 0.516634 
 0.470280 
 0.594231 
 
 
 1 
 0.405769 
 0.639218 
 0.639218 
 0.423747 
 0.423747 
 0.636984 
 0.480335 
 0.553543 
 0.529720 
 0.474282 
 0.483366 
 0.529720 
 0.405769

## Expectation step

$$
r_{nk} = \frac{
            \alpha_k(\prod_{t_m \in d_n} q_{mk})(\prod_{t_m \not\in d_n} (1-q_{mk}))
            }{
            \sum_{k=1}^{K}\alpha_k(\prod_{t_m \in d_n} q_{mk})(\prod_{t_m \not\in d_n} (1-q_{mk}))
            }
$$


In [11]:
def prod(cluster_index, document_index):
    p1 = np.product([qmk(i, cluster_index) for i in range(len(vocabulary)) if M[document_index,i] == 1])
    p0 = np.product([1 - qmk(i, cluster_index) for i in range(len(vocabulary)) if M[document_index,i] == 0])
    return A[cluster_index] * p1 * p0

def expectation(R):
    for document_index in range(N):
        total = sum([prod(c, document_index) for c in range(K)])
        for cluster_index in range(K):
            R[document_index, cluster_index] = prod(cluster_index, document_index) / total
    return R / R.sum(axis=1).reshape(-1, 1)

In [12]:
Rdf = pd.DataFrame(R)
display(Rdf)

R = expectation(R)

Rdf = pd.DataFrame(R)
display(Rdf)

,0,1
0,0.330593,0.669407
1,0.487531,0.512469
2,0.626290,0.373710
3,0.326332,0.673668
4,0.538564,0.461436


,0,1
0,0.258404,0.741596
1,0.766239,0.233761
2,0.915168,0.084832
3,0.001682,0.998318
4,0.037992,0.962008


## Put things together

In [13]:
class EMClustering(object):

    def __init__(self, corpus, k=2):
        self.corpus = corpus
        self.vocabulary = list(set([x for y in self.corpus for x in y]))
        self.M = np.array([[1 if x in y else 0 for x in self.vocabulary] for y in self.corpus])
        self.N, self.K, self.m = len(self.corpus), k, len(self.vocabulary)
        self.A = np.zeros(self.K)
        self.Q = np.zeros((self.m, self.K))
        # Init R
        self.R = np.random.uniform(size=(self.N, self.K))
        self.R = self.R / self.R.sum(axis=1).reshape(-1, 1) 

    def qmk(self, token_index, cluster_index):
        num = (self.R[:,cluster_index] * self.M[:,token_index]).sum()
        den = self.R[:,cluster_index].sum()
        return num / den

    def ak(self, cluster_index):
        return self.R[:,cluster_index].sum() / self.N
    
    def maxstep(self):
        for cluster_index in range(self.K):
            for token_index in range(self.m):
                self.Q[token_index, cluster_index] = self.qmk(token_index, cluster_index)
            self.A[cluster_index] = self.ak(cluster_index)
        self.Q = self.Q / self.Q.sum(axis=1).reshape(-1, 1)

    def prod(self, cluster_index, document_index):
        p1 = np.product([self.qmk(i, cluster_index) for i in range(self.m) if self.M[document_index,i] == 1])
        p0 = np.product([1 - self.qmk(i, cluster_index) for i in range(self.m) if self.M[document_index,i] == 0])
        return self.A[cluster_index] * p1 * p0

    def expectation(self):
        for document_index in range(self.N):
            total = sum([self.prod(c, document_index) for c in range(self.K)])
            for cluster_index in range(self.K):
                self.R[document_index, cluster_index] = self.prod(cluster_index, document_index) / total
    
    def fit(self, iterations=10):
        self.maxstep()
        self.expectation()

## Try with different corpora

In [14]:
corpus = [
    "sugar milk eggs butter".split(),
    "vanilla eggs apple peach sugar".split(),
    "peach apple sugar milk".split(),
    "meat chicken olive salt butter".split(),
    "meat beef salt pepper".split()
]

In [19]:
EM = EMClustering(corpus)
EM.fit(iterations=10)

In [20]:
Adf = pd.DataFrame(EM.A)
Qdf = pd.DataFrame(EM.Q, index=EM.vocabulary).T
A_styler = Adf.style.set_table_attributes("style='display:inline'").set_caption('A')
Q_styler = Qdf.style.set_table_attributes("style='display:inline'").set_caption('Q')
display_html(A_styler._repr_html_() + Q_styler._repr_html_(), raw=True)
Rdf = pd.DataFrame(EM.R)
display(round(Rdf, 2))

A 
 
 
   
 0 
 
 
 
 
 0 
 0.401649 
 
 
 1 
 0.598351 
 
 
 

 
 Q 
 
 
   
 peach 
 chicken 
 olive 
 beef 
 pepper 
 butter 
 sugar 
 eggs 
 salt 
 vanilla 
 milk 
 meat 
 apple 
 
 
 
 
 0 
 0.256165 
 0.797499 
 0.797499 
 0.640705 
 0.640705 
 0.639873 
 0.327018 
 0.250330 
 0.721748 
 0.005762 
 0.463433 
 0.721748 
 0.256165 
 
 
 1 
 0.743835 
 0.202501 
 0.202501 
 0.359295 
 0.359295 
 0.360127 
 0.672982 
 0.749670 
 0.278252 
 0.994238 
 0.536567 
 0.278252 
 0.743835

,0,1
0,0.06,0.94
1,0.00,1.00
2,0.00,1.00
3,1.00,0.00
4,1.00,0.00


In [21]:
from collections import defaultdict

In [22]:
cluster_description = defaultdict(list)
for i, cluster in enumerate(np.argmax(EM.Q, axis=1)):
    cluster_description[cluster].append(EM.vocabulary[i])
for cluster, words in cluster_description.items():
    print(cluster, words)

1 ['peach', 'sugar', 'eggs', 'vanilla', 'milk', 'apple']
0 ['chicken', 'olive', 'beef', 'pepper', 'butter', 'salt', 'meat']


In [26]:
cluster_docs = defaultdict(list)
for i, cluster in enumerate(np.argmax(EM.R, axis=1)):
    cluster_docs[cluster].append(corpus[i])
for cluster, docs in cluster_docs.items():
    print(cluster)
    for doc in docs:
        print('-', " ".join(doc))

1
- sugar milk eggs butter
- vanilla eggs apple peach sugar
- peach apple sugar milk
0
- meat chicken olive salt butter
- meat beef salt pepper
